In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[Problem 1] Learning / Estimation

In [ ]:
!git clone https://github.com/zhixuhao/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 44.92 MiB | 16.89 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Checking out files: 100% (253/253), done.


In [ ]:
cd drive/MyDrive/unet

/content/drive/MyDrive/unet


In [ ]:
!python main.py

2021-05-20 09:17:36.841495: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-20 09:17:38.590251: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-20 09:17:38.594402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-20 09:17:38.609126: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-20 09:17:38.609201: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (74e7f9501ef0): /proc/driver/nvidia/version does not exist
2021-05-20 09:17:38.609992: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/traini

In [ ]:
from model import *
from data import *

data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')

myGene = trainGenerator(20,
                        'data/membrane/train',
                        'image',
                        'label',
                        data_gen_args,
                        save_to_dir = None)

model = unet()
model_checkpoint = ModelCheckpoint('unet_salt.hdf5',
                                   monitor='loss',
                                   verbose=1,
                                   save_best_only=True)

model.fit_generator(myGene,
                    steps_per_epoch=10,
                    epochs=5,
                    callbacks=[model_checkpoint])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/5
10/10 [==============================] - 760s 73s/step - loss: 0.5088 - accuracy: 0.7813

Epoch 00001: loss improved from inf to 0.50146, saving model to unet_salt.hdf5
Epoch 2/5
10/10 [==============================] - 753s 73s/step - loss: 0.4225 - accuracy: 0.7881

Epoch 00002: loss improved from 0.50146 to 0.41002, saving model to unet_salt.hdf5
Epoch 3/5
10/10 [==============================] - 752s 72s/step - loss: 0.3716 - accuracy: 0.8151

Epoch 00003: loss improved from 0.41002 to 0.36544, saving model to unet_salt.hdf5
Epoch 4/5
10/10 [==============================] - 756s 73s/step - loss: 0.3488 - accuracy: 0.8407

Epoch 00004: loss improved from 0.36544 to 0.34569, saving model to unet_salt.hdf5
Epoch 5/5
10/10 [==============================] - 768s 74s/step - loss: 0.3345 - accuracy: 0.8495

Epoch 00005: loss improved from 0.34569 to 0.33423, saving model to unet_salt.hdf5


In [ ]:
gene = testGenerator('data/membrane/test')
unet = unet()
unet.load_weights('unet_salt.hdf5')
pred = model.predict_generator(gene,10,verbose=1)
saveResult("data/membrane/test",pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


10/10 [==============================] - 15s 1s/step


[Problem 2] Code reading

- this paper, presents a network and training strategy that relies on the strong
use of data augmentation to use the available annotated samples more
efficiently.

- This network is based on a fully convolutional network, it also uses many layers and in this network pooling operations are replaced with upsampling operators